# FreshHarvest Model Explainability Analysis

This notebook provides explainability analysis for the FreshHarvest fruit freshness classification model including:
- Grad-CAM visualizations
- Feature importance analysis
- Layer activation visualizations
- Model interpretation techniques
- Decision boundary analysis

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# Import custom modules
from cvProject_FreshHarvest.utils.common import read_yaml, setup_logging
from cvProject_FreshHarvest.models.cnn_models import FreshHarvestCNN

# Setup
setup_logging()
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 1. Load Model and Configuration

In [ ]:
# Load configuration
config = read_yaml('../config/config.yaml')
print("Configuration loaded:")
print(f"- Image size: {config['data']['image_size']}")
print(f"- Number of classes: {config['data']['num_classes']}")

# Define class names
CLASS_NAMES = [
    'F_Banana', 'F_Lemon', 'F_Lulo', 'F_Mango', 'F_Orange', 'F_Strawberry', 'F_Tamarillo', 'F_Tomato',
    'S_Banana', 'S_Lemon', 'S_Lulo', 'S_Mango', 'S_Orange', 'S_Strawberry', 'S_Tamarillo', 'S_Tomato'
]

print(f"\nClass names: {CLASS_NAMES}")

In [ ]:
# Load trained model
def load_trained_model(model_path):
    """Load a trained model from file."""
    try:
        model = keras.models.load_model(model_path)
        print(f"✅ Model loaded successfully from {model_path}")
        print(f"Model input shape: {model.input_shape}")
        print(f"Model output shape: {model.output_shape}")
        return model
    except Exception as e:
        print(f"❌ Failed to load model: {e}")
        return None

# Try to load different model files
model_paths = [
    '../models/trained/best_model.h5',
    '../models/best_hypertuned_model.h5',
    '../models/checkpoints/best_model_20250618_100126.h5'
]

model = None
for path in model_paths:
    if os.path.exists(path):
        model = load_trained_model(path)
        if model is not None:
            print(f"Using model: {path}")
            break

if model is None:
    print("⚠️ No trained model found. Creating a new lightweight model for demonstration.")
    cnn_builder = FreshHarvestCNN('../config/config.yaml')
    model = cnn_builder.create_lightweight_cnn()
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

# Display model architecture
print("\n📋 Model Architecture Summary:")
model.summary()

## 2. Grad-CAM Implementation

In [ ]:
# Grad-CAM implementation
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """Generate Grad-CAM heatmap."""
    
    # Create a model that maps the input image to the activations of the last conv layer
    # as well as the output predictions
    grad_model = keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    # Compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    
    # Gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)
    
    # Vector of mean intensity of the gradient over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    # Normalize the heatmap between 0 & 1 for visualization
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def find_last_conv_layer(model):
    """Find the last convolutional layer in the model."""
    for layer in reversed(model.layers):
        if 'conv' in layer.name.lower():
            return layer.name
    return None

# Find the last convolutional layer
last_conv_layer_name = find_last_conv_layer(model)
print(f"Last convolutional layer: {last_conv_layer_name}")

if last_conv_layer_name is None:
    print("⚠️ No convolutional layer found in the model")
else:
    print(f"✅ Using layer '{last_conv_layer_name}' for Grad-CAM analysis")

## 3. Load Sample Images for Analysis

In [ ]:
# Load sample images for explainability analysis
def load_sample_images(data_dir, num_samples=5):
    """Load sample images from each class."""
    
    sample_images = []
    sample_labels = []
    sample_paths = []
    
    try:
        for class_idx, class_name in enumerate(CLASS_NAMES[:8]):  # First 8 classes
            class_dir = os.path.join(data_dir, class_name)
            if os.path.exists(class_dir):
                image_files = [f for f in os.listdir(class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                
                # Take first available image
                if image_files:
                    img_path = os.path.join(class_dir, image_files[0])
                    
                    # Load and preprocess image
                    img = load_img(img_path, target_size=tuple(config['data']['image_size']))
                    img_array = img_to_array(img)
                    img_array = np.expand_dims(img_array, axis=0)
                    img_array = img_array / 255.0  # Normalize
                    
                    sample_images.append(img_array)
                    sample_labels.append(class_idx)
                    sample_paths.append(img_path)
                    
                    print(f"✅ Loaded sample for {class_name}")
                else:
                    print(f"⚠️ No images found for {class_name}")
            else:
                print(f"⚠️ Directory not found: {class_dir}")
    
    except Exception as e:
        print(f"❌ Error loading sample images: {e}")
        # Create dummy data for demonstration
        print("Creating dummy sample images for demonstration...")
        for i in range(4):
            dummy_img = np.random.random((1, 224, 224, 3))
            sample_images.append(dummy_img)
            sample_labels.append(i)
            sample_paths.append(f"dummy_image_{i}.jpg")
    
    return sample_images, sample_labels, sample_paths

# Load sample images
sample_dirs = ['../data/processed/test', '../data/processed/val', '../data/raw']
sample_images, sample_labels, sample_paths = None, None, None

for data_dir in sample_dirs:
    if os.path.exists(data_dir):
        print(f"\nTrying to load samples from: {data_dir}")
        sample_images, sample_labels, sample_paths = load_sample_images(data_dir)
        if sample_images:
            break

if not sample_images:
    print("\n⚠️ No sample images found. Creating dummy data for demonstration.")
    sample_images = [np.random.random((1, 224, 224, 3)) for _ in range(4)]
    sample_labels = [0, 1, 2, 3]
    sample_paths = [f"dummy_image_{i}.jpg" for i in range(4)]

print(f"\n✅ Loaded {len(sample_images)} sample images for analysis")